In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/Capstone

/content/gdrive/MyDrive/Capstone


# Loading the dataset

In [ ]:
import pandas
import numpy as np

df = pandas.read_csv('data.csv', index_col='NUM')

In [ ]:
del df["NAME"]
del df["COUNTRY"]

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score

data, test = train_test_split(
    df,
    test_size = 0.2,
    random_state = 5)

print(data.shape)
print(test.shape)

(41, 6)
(11, 6)


In [ ]:
X = data.loc[:,['Mine Annual Production (Million Tonne)',
       'Stripping Ratio', 'Mill Annual Production (Thousand Tonne)',
       'Reserve Mean Grade % Cu EQU.', 'LOM']]
y = data["CAPEX US$ millions"]

In [ ]:
num_folds = 6

kf = KFold(n_splits=num_folds, shuffle=True, random_state=5)

for c in kf.split(data):
    print(c)

(array([ 0,  1,  2,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       21, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]), array([ 3,  4,  6, 20, 22, 23, 24]))
(array([ 1,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39]), array([ 0,  2,  5, 26, 28, 34, 40]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17,
       20, 21, 22, 23, 24, 26, 27, 28, 30, 31, 32, 33, 34, 35, 37, 38, 40]), array([10, 18, 19, 25, 29, 36, 39]))
(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 14, 15, 16, 17, 18, 19,
       20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 34, 35, 36, 37, 38, 39, 40]), array([ 1, 11, 13, 21, 31, 32, 33]))
(array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 13, 14, 16, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 28, 29, 31, 32, 33, 34, 35, 36, 38, 39, 40]), array([ 7, 12, 15, 17, 27, 30, 37]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7, 10, 11, 12

# K-fold Training, Validation, Optimisation

In [ ]:
# Metrics:
#   RMSE -> Root Mean Squared Error
#   R2 -> Coefficient of Correlation
#   MAE -> Mean Absolute Error
#   APE -> Absolute Error

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()
scores = cross_val_score(rf, X, y, cv=kf)
print(scores)
print(np.mean(scores))


# average the scores for the model
# then use scores for the optimisation algorithms

[ 0.60228018  0.38767531 -1.86661295  0.17676743  0.49824437  0.54742912]
0.05763057812433733


In [ ]:
# ANN

In [ ]:
# SVM

In [ ]:
# CART Tree (decision tree)

In [ ]:
# Also look at what hyperparameters we can optimise for each.

# Testing

In [ ]:
#use optimal hyperparameters to train model, then test

To add later on if time:  
-preprocessing (outliers, etc.)  
-ensemble methods (ANN + SVM)